# Проект. Рекомендация тарифов

## Глава 0. Входные данные
### Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.

Необходимо построить модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75 и проверить *accuracy* на тестовой выборке.

#### Путь к файлу:
- /datasets/users_behavior.csv  

#### Описание данных таблицы
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

#### План действий
1 - Ознакомление с данными.  
2 - Разделить исходные данные на обучающую, валидационную и тестовую выборки.  
3 - Исследование качество разных моделей, меняя гиперпараметры.  
4 - Проверка качество модели на тестовой выборке.  
5 - Дополнительно: проверка модели на вменяемость.  

## Глава 1. Открытие файла с данными и изучение общей информации

Вызовем все необходимые библиотеки для реализации проекта

In [1]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

Прочитаем файл, создадим ДатаФрейм, выведем информацию и первые 10 строк таблицы

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Выводы по главе 1:

> Предобработка выполнена ранее. Пропусков нет, данные имеют нужный формат. Что позволяет перейти сразу к исследованиям

## Глава 2. Разделение исходные данные на обучающую, валидационную и тестовую выборки.

Разделим датафрейм на 3 части: обучающую, валидационную и тестовую выбороки в пропорциях 3:1:1, используя функцию `train_test_split` из библиетеки *sklearn.model_selection*  
Разделим сначала `df` на `df_train` и `df_valid_test`, а потом `df_valid_test` на `df_valid` и `df_test`

In [3]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

Выведем размеры обучающей, валидационной и тестовой выборок на экран и оценим правильность деления на части

In [4]:
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Разделим каждую выборку на `features` -  признаки и `target` — целевой признак.

In [5]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_valid.drop(['is_ultra'], axis=1)
target_test = df_valid['is_ultra']

### Выводы по главе 2:

> Разделили все данные на 3 части. Обучающая - для обучения моделей, Валидационная - для настройки гиперпараметров модели, Тестовая - для окончательной проверки модели на неизвестных данных и оценки качества модели

## Глава 3. Исследование качества разных моделей

Данная задача относиться к задачам классификации, в нашем случае бинарной классификации.  
Рассмотрим 3 модели:  
 - **Решающее дерево** DecisionTreeClassifier из библиотеки *sklearn.tree*
 - **Случайный лес** RandomForestClassifier из библиотеки *sklearn.ensemble*
 - **Логистическая регрессия** LogisticRegression из библиотеки *sklearn.linear_model*  
 
 Сразу введем псевдослучайность для алгоритма обучения, используя генератор псевдослучайных чисел `random_state`. Таким образом модель будет воспринимать данные как случайные, для нас это необходимо , чтобы результаты неизменно получались одинаковыми (другими словами, повторить удачный эксперимент).

### 1. Решающее дерево - DecisionTreeClassifier

Создадим модель и запишем в переменную **model_decision_tree**, обучим модель используя метод `fit()` на признаках **features_train** и целевом признаке **target_train**.

In [6]:
model_decision_tree = DecisionTreeClassifier(random_state=12345)
model_decision_tree.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

Чтобы предсказать ответы, вызовем метод `predict()`и передать ему таблицу с признаками новых объектов **features_valid**.

In [7]:
tree_valid_predictions = model_decision_tree.predict(features_valid)

Посчитаем оценку точности предсказания `accuracy_score` для модели, сравнив предсказания **tree_valid_predictions** и правильные ответы **target_valid** в валидационной выборке.

In [8]:
tree_accuracy = accuracy_score(target_valid, tree_valid_predictions)
tree_accuracy

0.713841368584759

Меняя гиперпараметры модели, посмотрим как меняется оценка качества **accuracy**

`max_depth` - максимальная глубина дерева. Переберем в цикле значения от 1 до 20:

In [9]:
for depth in range(1,21):
    model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_decision_tree.fit(features_train, target_train)
    tree_valid_predictions = model_decision_tree.predict(features_valid)
    print(depth,':', accuracy_score(target_valid, tree_valid_predictions))

1 : 0.7542768273716952
2 : 0.7822706065318819
3 : 0.7853810264385692
4 : 0.7791601866251944
5 : 0.7791601866251944
6 : 0.7838258164852255
7 : 0.7822706065318819
8 : 0.7791601866251944
9 : 0.7822706065318819
10 : 0.7744945567651633
11 : 0.7620528771384136
12 : 0.7620528771384136
13 : 0.7558320373250389
14 : 0.7589424572317263
15 : 0.7465007776049767
16 : 0.7340590979782271
17 : 0.7356143079315708
18 : 0.7309486780715396
19 : 0.7278382581648523
20 : 0.7216174183514774


>Качество модели падает при большом увеличении величины глубины дерева, т.к. модель решающее дерево склонно к переобучению. Оптимальным, в данном случае,  `max_depth=3`

Изменим критерий Джини на энтропию:

In [10]:
model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=3, criterion='entropy')
model_decision_tree.fit(features_train, target_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=12345)

In [11]:
tree_valid_predictions = model_decision_tree.predict(features_valid)

In [12]:
tree_accuracy = accuracy_score(target_valid, tree_valid_predictions)
tree_accuracy

0.7853810264385692

>Качество модели не изменилось

### 2. Случайный лес - RandomForestClassifier

Создадим модель случайный лес(с количеством деревьев `n_estimators=5`) и запишем в переменную **model_random_forest**, обучим модель используя метод `fit()` на признаках **features_train** и целевом признаке **target_train**.

In [13]:
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=5)
model_random_forest.fit(features_train, target_train)

RandomForestClassifier(n_estimators=5, random_state=12345)

Чтобы предсказать ответы, вызовем метод `predict()`и передать ему таблицу с признаками новых объектов **features_valid**.

In [14]:
forest_valid_predictions = model_random_forest.predict(features_valid)

Посчитаем оценку точности предсказания `accuracy_score` для модели, сравнив предсказания **forest_valid_predictions** и правильные ответы **target_valid** в валидационной выборке.

In [15]:
forest_accuracy = accuracy_score(target_valid, forest_valid_predictions)
forest_accuracy

0.749611197511664

Меняя гиперпараметры модели, посмотрим как меняется оценка качества **accuracy**

Изменим критерий Джини на энтропию:

In [16]:
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=5, criterion='entropy')
model_random_forest.fit(features_train, target_train)
forest_valid_predictions = model_random_forest.predict(features_valid)
forest_accuracy = accuracy_score(target_valid, forest_valid_predictions)
forest_accuracy

0.7620528771384136

>Качество модели изменилось примерно на 1%

`n_estimators` - количество деревьев. Переберем в цикле значения от 5 до 100 с шагом 5:

In [17]:
for estimators in range(5,101,5):
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=estimators, criterion='entropy')
    model_random_forest.fit(features_train, target_train)
    forest_valid_predictions = model_random_forest.predict(features_valid)
    print(estimators,':', accuracy_score(target_valid, forest_valid_predictions))

5 : 0.7620528771384136
10 : 0.7853810264385692
15 : 0.7807153965785381
20 : 0.7838258164852255
25 : 0.7869362363919129
30 : 0.7947122861586314
35 : 0.7931570762052877
40 : 0.7947122861586314
45 : 0.7962674961119751
50 : 0.7993779160186625
55 : 0.7931570762052877
60 : 0.7916018662519441
65 : 0.7916018662519441
70 : 0.7884914463452566
75 : 0.7884914463452566
80 : 0.7916018662519441
85 : 0.7884914463452566
90 : 0.7900466562986003
95 : 0.7884914463452566
100 : 0.7869362363919129


>Качество модели наилучшее при `n_estimators=50`, делее качество не растет, плюс снижает скорость работы модели

`max_depth` - максимальная глубина дерева. Переберем в цикле значения от 1 до 20:

In [18]:
for depth in range(1,21):
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=50, criterion='entropy', max_depth=depth)
    model_random_forest.fit(features_train, target_train)
    forest_valid_predictions = model_random_forest.predict(features_valid)
    print(depth,':', accuracy_score(target_valid, forest_valid_predictions))

1 : 0.7589424572317263
2 : 0.7869362363919129
3 : 0.7884914463452566
4 : 0.7884914463452566
5 : 0.7993779160186625
6 : 0.80248833592535
7 : 0.80248833592535
8 : 0.7993779160186625
9 : 0.8009331259720062
10 : 0.7947122861586314
11 : 0.7931570762052877
12 : 0.7993779160186625
13 : 0.7900466562986003
14 : 0.7884914463452566
15 : 0.7900466562986003
16 : 0.7962674961119751
17 : 0.7993779160186625
18 : 0.7900466562986003
19 : 0.7962674961119751
20 : 0.7900466562986003


>Качество модели не растет при увеличении величины глубины дерева. Оптимальным, в данном случае,  `max_depth=6`

`min_samples_split` - гиперпараметр запрещает создавать узлы, в которые попадает слишком мало объектов обучающей выборки.     Переберем в цикле значения от 2 до 10:

In [19]:
for sample in range(2,11):
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=50, criterion='entropy', max_depth=6, min_samples_split=sample)
    model_random_forest.fit(features_train, target_train)
    forest_valid_predictions = model_random_forest.predict(features_valid)
    print(sample,':', accuracy_score(target_valid, forest_valid_predictions))

2 : 0.80248833592535
3 : 0.80248833592535
4 : 0.8009331259720062
5 : 0.8040435458786936
6 : 0.7993779160186625
7 : 0.7993779160186625
8 : 0.8040435458786936
9 : 0.7993779160186625
10 : 0.80248833592535


>Качество модели оптимально при `min_samples_split=5`

Запишем все гиперпараметры Случайного леса и выведем итоговое качество модели:  
`n_estimators=50`, `criterion='entropy'`, `max_depth=6`, `min_samples_split=5`

In [20]:
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=50, criterion='entropy', max_depth=6, min_samples_split=5)
model_random_forest.fit(features_train, target_train)
forest_valid_predictions = model_random_forest.predict(features_valid)
accuracy_score(target_valid, forest_valid_predictions)

0.8040435458786936

### 3. Логистическая регрессия - LogisticRegression

Создадим модель Логистическая регрессия и запишем в переменную **logistic_model**, обучим модель используя метод `fit()` на признаках **features_train** и целевом признаке **target_train**.

In [21]:
logistic_model = LogisticRegression(random_state=12345)
logistic_model.fit(features_train, target_train)

LogisticRegression(random_state=12345)

Чтобы предсказать ответы, вызовем метод `predict()`и передать ему таблицу с признаками новых объектов **features_valid**.

In [22]:
logistic_valid_predictions = logistic_model.predict(features_valid)

Посчитаем оценку точности предсказания `accuracy_score` для модели, сравнив предсказания **logistic_valid_predictions** и правильные ответы **target_valid** в валидационной выборке.

In [23]:
logistic_accuracy = accuracy_score(target_valid, logistic_valid_predictions)
logistic_accuracy

0.7107309486780715

Меняя гиперпараметры модели, посмотрим как меняется оценка качества **accuracy**  
Данная модель имеет мало гиперпараметров для настройки, изменим `solver='liblinear'`, `warm_start=True`, `multi_class='ovr'`

In [24]:
logistic_model = LogisticRegression(random_state=12345, solver='liblinear', warm_start=True, multi_class='ovr')
logistic_model.fit(features_train, target_train)
logistic_valid_predictions = logistic_model.predict(features_valid)
logistic_accuracy = accuracy_score(target_valid, logistic_valid_predictions)
logistic_accuracy

0.7542768273716952

> Качество модели не стало лучше

### Выводы по главе 3:

> На качество обученной модели проверили Решающее дерево, Случайный лес и Логистическую регрессию. Лучшие показатели, с учетом всех настроек у Случайного леса, что не удивительно, использую сразу несколько независимых деревьев и определяя результат голосованием получается самое точное предсказание, но скорость работы становиться медленее, чем больше деревьев в лесу.  
Далее будем использовать модель Случайный лес

## Глава 4. Проверка качество модели на тестовой выборке.

Так как модель определена и ее гиперпараметры выяснены, объеденим обучающую и валидационную выборку, используя метод `.append`, для того чтобы обучить модель еще немного лучше.

In [25]:
df_train_valid = df_train.append(df_valid, ignore_index=True)
df_train_valid.shape

(2571, 5)

>Обучающая выборка стала на 20% больше.

Разделим обучающую выборку `df_train_valid` на признаки `features_train_valid` и целевой признак `target_train_valid`

In [26]:
features_train_valid = df_train_valid.drop(['is_ultra'], axis=1)
target_train_valid = df_train_valid['is_ultra']

Заново обучим модель Случайный лес `model_random_forest` с настроенными гиперпараметрами на расширенной выборке:

In [27]:
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=50, criterion='entropy', max_depth=6, min_samples_split=5)
model_random_forest.fit(features_train_valid, target_train_valid)

RandomForestClassifier(criterion='entropy', max_depth=6, min_samples_split=5,
                       n_estimators=50, random_state=12345)

Чтобы предсказать ответы, вызовем метод `predict()`и передать ему таблицу с признаками новых объектов **features_test**.

In [28]:
forest_valid_predictions = model_random_forest.predict(features_test)

Посчитаем оценку точности предсказания `accuracy_score` для модели, сравнив предсказания **forest_valid_predictions** и правильные ответы **target_test** в тестовой выборке.

In [29]:
forest_accuracy = accuracy_score(target_test, forest_valid_predictions)
forest_accuracy

0.8211508553654744

Так же оценку качества модели можно получить вызвав функцию `.score()`

In [30]:
model_random_forest.score(features_test, target_test)

0.8211508553654744

Вычислим значение качества модели на обучающей выборке: 

In [31]:
model_random_forest.score(features_train_valid, target_train_valid)

0.8245818747569039

> Качество на обучающей выборке и тестовой почти равны, что говорит о отсутствии переобучения у модели.

### Выводы по главе 4:

> На тестовой выборке модель показала точность 82%, что говорит о хорошем качестве предсказания результатов на незнакомых данных. Так же модель не переучена, т.к. на тестовой и обучающей выборке результаты почти равны

## Глава 5. Дополнительно: проверка модели на вменяемость.

Для того, чтобы получить «случайные» результаты, воспользуемся DummyClassifier. Полученные им результаты абсолютно случайные.  
Стратегию выставим `strategy='uniform'` как самую случайную

In [32]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)

Обучим модель на наших данных и посчитаем точность предсказания: 

In [33]:
dummy_model.fit(features_train_valid, target_train_valid)
dummy_model.score(features_test, target_test)

0.7060653188180405

Оценка предсказания модели равна 0.5. То есть с вероятностью 50/50 она предсказывает результат

### Выводы по главе 5:

> Результаты метрик для модели Случайный лес и DummyClassifier, отличаются на 30%. Это говорит о том, что обученная модель работает лучше, чем «абсолютно случайные результаты».

## Глава 6. Общий вывод

- Модель Случайный лес показала самые лучшие результаты из 3 моделей.
- Используя десятки Решающих независимых деревьев, модель выбирает оптимальный вариант.
- Большое количество настроек гиперпараметров позволяет усовершенствовать модель
- Полученные значения качества модели в 82% говорит о хорошем предсказании.  
- Построенная модель для задачи классификации, сможет подобрать для пользователей, пользующихся архивными тарифами, более подходящий тариф («Смарт» или «Ультра»).  
- Проанализировав поведение клиентов: тратах минут, сообщений и интернет трафика, модель в 4 из 5 случаев предскажет верный тариф для пользователя. 
- Модель подходит как система "Рекомендации тарифов"

In [34]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))